Task 3 using Clustering:

In this task you will implement functions to enable group behaviour recognition. By analysing the image sequence (video) you should:

a) Provide real-time information on how many pedestrians walk in groups (of more than 1 person), and how many pedestrians walk alone

b) Detect and show (using a bounding box or other means of pointing) occurrences of group formation (eg. two people meet and stay together for more than one frame, two people meet up and walk together, etc) and group destruction (when members in a group split)

In [0]:
import numpy as np
import os
import cv2
from scipy.spatial import distance
confThreshold = 0.75
nmsThreshold = 0.40
inpWidth = 576
inpHeight = 768

#Load names of classes and turn that into a list
classesFile = "coco.names"
classes = None

with open(classesFile,'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

#Model configuration
modelConf = 'yolov3.cfg'
modelWeights = 'yolov3.weights'


def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

#Set up the net
net = cv2.dnn.readNetFromDarknet(modelConf, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
fn = '000000' 
for count in range(1,len(os.listdir('sequence'))):
    a = ('sequence/'+fn[len(str(count)):]+str(count)+'.jpg')
    b = ('task_3_out_images_clustering/'+fn[len(str(count)):]+str(count)+'.jpg')
    frame = cv2.imread(a)#cv2.imread('images/000051.jpg')#cv2.VideoCapture(0)
    rects = []
    centroidList = []
    blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop = False)
    net.setInput(blob)
    outs = net.forward (getOutputsNames(net))
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]
    classIDs = []
    confidences = []
    boxes = []
    status = "People in Group"
    for out in outs:
        for detection in out:            
            scores = detection [5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > confThreshold:
                centerX = int(detection[0] * frameWidth)
                centerY = int(detection[1] * frameHeight)
                width = int(detection[2]* frameWidth)
                height = int(detection[3]*frameHeight )
                left = int(centerX - width/2)
                top = int(centerY - height/2)
                classIDs.append(classID)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
    indices = cv2.dnn.NMSBoxes (boxes,confidences, confThreshold, nmsThreshold )
    
    for i in indices:
        i = i[0]
        box = boxes[i]
        startX = box[0]
        startY = box[1]
        endX = box[2] + box[0]
        endY = box[3] + box[1]
        classId = classIDs[i]
        conf = confidences[i]
        label = '%.2f' % conf
        if classes:
            assert (classId < len(classes))
            label = '%s:%s' % (classes[classId], label)
        if label.startswith('person'):
            rects.append((startX, startY, endX, endY))
            centerX = startX + (endX - startX)/2
            centerY = endY + (startY - endY)/2
            centroidList.append([int(centerX), int(centerY)])
            
    print(a)
    # Clustering the centriod values of each predestian values for getting the group of predestians and persons walking alone
    clusters = hcluster.fclusterdata(centroidList,t=0.73) #need to adjust threshold
    clusters = np.asarray(clusters)
    
    # To mark group and individuals
    def draw_dot_single(indexList):
        for i in indexList:
            cv2.circle(frame, (int(centroidList[i][0]),int(centroidList[i][1])), 3, (0, 0, 255), 3)
            
    def draw_dot_single_1(centroidList):
        for i in centroidList:
            cv2.circle(frame, (int(i[0]),int(i[1])), 3, (0, 0, 255), 3)
            
    def draw_line(cList,indexList):
        groupCentroid = np.mean(cList, axis=0)   
#         cv2.circle(frame, (int(groupCentroid[0]),int(groupCentroid[1])), 3, (0, 0, 255), 3)
        for i in indexList:
            cv2.circle(frame, (int(centroidList[i][0]),int(centroidList[i][1])), 3, (0, 0, 255), 3)
            for j in indexList:
                cv2.line(frame, (int(centroidList[i][0]),int(centroidList[i][1])),
                     (int(centroidList[j][0]),int(centroidList[j][1])), (0, 255, 0), 2)
            
        
    groups = dict(zip(*np.unique(clusters, return_counts=True)))
    single=0
    n=1
    count = set()
    info = [("Status", status)]
    if len(groups) ==1:
        # print("No of single pedestrians :",len(clusters))
        info.append(("count for group" , single))
        info.append(("No of single pedestrians" , len(clusters)))
        draw_dot_single_1(centroidList)
    else:

        for k,v in groups.items():
            if v==1:
                single+=1
                indexList = np.where(clusters == k)[0]
                draw_dot_single(indexList)
            else:
                info.append((f"count for group {n}" , v))
                n+=1
                count.add(v)

            indexList = np.where(clusters == k)[0]
            draw_line([centroidList[i] for i in indexList], indexList)
        
        info.append(("No of single pedestrians" , single))
    rect_x,rect_y,rect_w,rect_h = 0,0,250,80
    cv2.rectangle(frame, (rect_x, rect_x), (rect_x + rect_w, rect_y + rect_h), (0,0,0), -1)
#     # loop over the info tuples and draw them on our frame
    for (i, (k, v)) in enumerate(info):
        text = "{}: {}".format(k, v)
        cv2.putText(frame, text, (10, ((i * 20) + 15)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.imwrite(b, frame)
    # break
cv2.destroyAllWindows()

In [0]:
import os
import cv2
img_array = []
fn = '000000' 
for count in range(1,len(os.listdir('task_3_out_images_clustering'))):
    a = ('task_3_out_images_clustering/'+fn[len(str(count)):]+str(count)+'.jpg')
    img = cv2.imread(a)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter('task_3_clustering.avi',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()